This notebook generates multiple AI-refined versions of ELL essays
using prompt-based transformations and a large language model.

The outputs are stored in a structured benchmark format for analysis.


In [1]:
!pip install openai tqdm pyyaml -q

import os
import sys
import json
import time
from datetime import datetime, timezone
from tqdm.auto import tqdm
from google.colab import drive
from google.colab import userdata
from openai import OpenAI

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# IMPORTANT: UPDATE THIS WITH YOUR OWN FOLDER PATH
PROJECT_FOLDER = '/content/drive/MyDrive/Fırat Projeler/Detecting AI Influence in Student Writing: Toward Reliable and Interpretable Classifiers/feedback-prize-english-language-learning'

sys.path.append(PROJECT_FOLDER)
os.chdir(PROJECT_FOLDER)


Mounted at /content/drive


In [2]:
# Import the instructor's module
try:
    import essay_prompts_v1 as prompt_utils
    print(f"✅ Success: 'essay_prompts_v1.py' and the YAML file loaded.")
except ImportError:
    print("❌ ERROR: 'essay_prompts_v1.py' file not found!")
    print(f"Please make sure the files are uploaded to the '{PROJECT_FOLDER}' folder.")
    sys.exit()
except FileNotFoundError:
    print("❌ ERROR: 'ai_influence_v1.yaml' file not found!")
    sys.exit()

# API Settings
PROVIDER = "GEMINI"
MODEL_NAME = "gemini-2.5-flash"
API_KEY_NAME = "GEMINI_Efe"  # Name in Colab Secrets
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

# OpenAI
#PROVIDER = "OPENAI"
#MODEL_NAME = "gpt-4o-mini"
#API_KEY_NAME = "OPENAI_API_KEY" # Secrets kısmındaki adı
#BASE_URL = None # OpenAI için varsayılan kalır

# DeepSeek (V3)
# PROVIDER = "DEEPSEEK"
# MODEL_NAME = "deepseek-chat"
# API_KEY_NAME = "DEEPSEEK_API_KEY"
# BASE_URL = "https://api.deepseek.com"


# Input / Output Files
INPUT_FILE = os.path.join(PROJECT_FOLDER, 'ell_balanced_1000.jsonl')
OUTPUT_FOLDER = os.path.join(PROJECT_FOLDER, f'Benchmark_Outputs/{PROVIDER}_{MODEL_NAME}')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
OUTPUT_FILE = os.path.join(OUTPUT_FOLDER, 'ELL_Benchmark_Generated.jsonl')

# Whatever number you enter, the code will process that many new text strings and stop automatically.
BATCH_LIMIT = 2
# BATCH_LIMIT = None  #(Open this to process everything, close the top one)

# Initialize API connection
try:
    api_key = userdata.get(API_KEY_NAME)
    if BASE_URL:
        client = OpenAI(api_key=api_key, base_url=BASE_URL)
    else:
        client = OpenAI(api_key=api_key)
    print(f"API connection ready: {PROVIDER}")
except Exception as e:
    print(f"API key error: {e}. Please check the Colab Secrets section.")


✅ Success: 'essay_prompts_v1.py' and the YAML file loaded.
API connection ready: GEMINI


In [3]:
def call_llm_with_messages(messages, retries=3):
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
                temperature=0.7,
                timeout=60
            )
            return response.choices[0].message.content, "OK"
        except Exception as e:
            wait_time = 5 if "rate limit" not in str(e).lower() else 20
            time.sleep(wait_time)
            if attempt == retries - 1:
                return "", f"API_ERROR: {str(e)}"


def process_essay_family(family):
    root_data = family.get('json_data', family)

    # Find the original text
    orig = next(
        (v for v in root_data.get('versions', []) if v['type'] == 'original'),
        None
    )
    if not orig:
        return None

    # Copy original data
    new_family_structure = root_data.copy()
    new_family_structure['versions'] = [orig]  # Keep only the original version

    # LOAD PROMPTS FROM FILE
    templates = prompt_utils.list_prompts()

    for tmpl in templates:
        try:
            messages = tmpl.build_chat_messages(essay_text=orig['text'])
        except ValueError as e:
            print(f"Prompt Error ({tmpl.id}): {e}")
            continue

        ai_text, status = call_llm_with_messages(messages)

        # Save the result
        new_family_structure['versions'].append({
            "type": tmpl.id,  # e.g., v1.ai_refined_grammar
            "text": ai_text if ai_text else "",
            "length_chars": len(ai_text) if ai_text else 0,
            "llm_model": f"{PROVIDER}:{MODEL_NAME}",
            "prompt_used_id": tmpl.id,
            "improvement_aspects": tmpl.aspects,
            "generated_at": datetime.now(timezone.utc).isoformat(),
            "status": status
        })

        # Short delay between API calls
        time.sleep(0.5)

    return new_family_structure


In [4]:
if __name__ == "__main__":
    print("-" * 50)
    print(f"Input File: {INPUT_FILE}")

    if not os.path.exists(INPUT_FILE):
        print(f"ERROR: Input file not found!")
    else:
        # Read file line by line
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            families = [json.loads(line) for line in f]

        # Identify already processed entries
        processed_ids = set()
        if os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        data = json.loads(line)
                        fid = data.get('family_id') or data.get('json_data', {}).get('family_id')
                        if fid:
                            processed_ids.add(fid)
                    except:
                        pass

        remaining = len(families) - len(processed_ids)
        print(f"Status: Total {len(families)} | Completed {len(processed_ids)} | Remaining {remaining}")

        if BATCH_LIMIT:
            print(f"TEST MODE: Only {BATCH_LIMIT} records will be processed.")

        print("-" * 50)

        counter = 0
        with open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
            for family in tqdm(families, desc="AI Generation in Progress"):

                # ID check
                fid = family.get('json_data', {}).get('family_id') or family.get('family_id')
                if fid in processed_ids:
                    continue

                # Limit check
                if BATCH_LIMIT and counter >= BATCH_LIMIT:
                    print(f"\nTarget limit ({BATCH_LIMIT}) reached. Stopping.")
                    break

                # Processing
                updated = process_essay_family(family)

                # Save result
                if updated:
                    f_out.write(json.dumps(updated, ensure_ascii=False) + '\n')
                    f_out.flush()
                    os.fsync(f_out.fileno())
                    counter += 1

        print(f"\nPROCESS COMPLETED.")
        print(f"Output File: {OUTPUT_FILE}")


--------------------------------------------------
Input File: /content/drive/MyDrive/Fırat Projeler/Detecting AI Influence in Student Writing: Toward Reliable and Interpretable Classifiers/feedback-prize-english-language-learning/ell_balanced_1000.jsonl
Status: Total 1000 | Completed 0 | Remaining 1000
TEST MODE: Only 2 records will be processed.
--------------------------------------------------


AI Generation in Progress:   0%|          | 0/1000 [00:00<?, ?it/s]


Target limit (2) reached. Stopping.

PROCESS COMPLETED.
Output File: /content/drive/MyDrive/Fırat Projeler/Detecting AI Influence in Student Writing: Toward Reliable and Interpretable Classifiers/feedback-prize-english-language-learning/Benchmark_Outputs/GEMINI_gemini-2.5-flash/ELL_Benchmark_Generated.jsonl


In [6]:
import json
import os

BAKILACAK_SIRA = 1  # Enter the desired record index here
def inspect_full_details_v2(file_path, target_index):
    if not os.path.exists(file_path):
        print(f"ERROR: File not found: {file_path}")
        return

    print(f"FILE: {file_path}")
    print(f"Retrieving record at index {target_index}...\n")

    found = False

    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f, 1):
            if i == target_index:
                try:
                    data = json.loads(line)
                    found = True

                    # Family ID
                    fid = data.get('family_id') or data.get('json_data', {}).get('family_id') or "NOT FOUND"
                    print("=" * 100)
                    print(f"🆔 FAMILY ID: {fid}  |  INDEX: {i}")
                    print("=" * 100)

                    # Iterate over versions
                    versions = data.get('versions', [])
                    for idx, v in enumerate(versions, 1):

                        # --- 1. METADATA ---
                        v_type = v.get('type', 'Unknown')
                        v_id = v.get('id', 'None')
                        v_date = v.get('generated_at', 'No Date')
                        v_model = v.get('llm_model', 'No Model')
                        v_len = v.get('length_chars', 0)
                        v_status = v.get('status', 'OK')

                        print(f"\n [{idx}] TYPE: {v_type}")
                        print(f"   ├─ Text ID       : {v_id}")
                        print(f"   ├─ Generated At : {v_date}")
                        print(f"   ├─ Model Used   : {v_model}")
                        print(f"   ├─ Length       : {v_len} characters")
                        print(f"   └─ Status       : {v_status}")

                        # --- 2. TEXT CONTENT ---
                        print("\n   TEXT CONTENT:")
                        print("   " + "-" * 90)
                        print(v.get('text', ""))
                        print("   " + "-" * 90)
                        print("\n")

                    break

                except Exception as e:
                    print(f"Data parsing error: {e}")

    if not found:
        print(f"Record {target_index} not found. (Total lines: {i})")
try:
    inspect_full_details_v2(OUTPUT_FILE, BAKILACAK_SIRA)
except NameError:
    print("OUTPUT_FILE is not defined.")

FILE: /content/drive/MyDrive/Fırat Projeler/Detecting AI Influence in Student Writing: Toward Reliable and Interpretable Classifiers/feedback-prize-english-language-learning/Benchmark_Outputs/GEMINI_gemini-2.5-flash/ELL_Benchmark_Generated.jsonl
Retrieving record at index 1...

🆔 FAMILY ID: 80D2FE30F665  |  INDEX: 1

 [1] TYPE: original
   ├─ Text ID       : None
   ├─ Generated At : No Date
   ├─ Model Used   : No Model
   ├─ Length       : 3027 characters
   └─ Status       : OK

   TEXT CONTENT:
   ------------------------------------------------------------------------------------------
Offering distance learning as an option for students to attend classes from home by way of online or video conferencing by schools is a good idea, but I take a position on this issue to say that, students would not benefit from being able to attend classes classes from home due to the following reasons.

First of all,some students when offered distance learning will not pay attention to their lesson